In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

import shared_utils
import branca
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Get and transform test data (Long Beach Transit, 170)

In [2]:
lbt_itp_id = 170

In [3]:
# %%sql -o lbt_positions

# # get all vehicle positions on selected dates, for the feed with itp id 170, and url number 0
# SELECT *
# FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
# WHERE _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-10_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-02_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-03_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-04_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-05_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-06_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-07_170_0.parquet"
# ORDER BY header_timestamp

In [4]:
# lbt_positions.to_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

In [5]:
lbt_positions = pd.read_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

#### Reformat timestamps

In [6]:
## set system time
os.environ['TZ'] = 'America/Los_Angeles'
time.tzset()
time.tzname

('PST', 'PDT')

In [7]:
def convert_ts(ts):    
    pacific_dt = dt.datetime.fromtimestamp(ts)
    # print(pacific_dt)
    return pacific_dt

In [8]:
lbt_positions.vehicle_timestamp = lbt_positions.vehicle_timestamp.apply(convert_ts)
lbt_positions.header_timestamp = lbt_positions.header_timestamp.apply(convert_ts)

In [9]:
pch_routes = ('171', '172', '173', '174', '175')

In [10]:
# min_date, max_date = ('2021-11-01', '2021-12-09')

# lbt_trips = (tbl.views.gtfs_schedule_fact_daily_trips()
#     # >> filter(_.calitp_extracted_at <= min_date, _.calitp_deleted_at > max_date)
#     >> filter(_.calitp_itp_id == lbt_itp_id)
#     >> filter(_.service_date == max_date)
#     >> filter(_.is_in_service == True)
#     >> select(_.trip_key, _.service_date)
#     >> inner_join(_, tbl.views.gtfs_schedule_dim_trips(), on = 'trip_key')
#     >> select(_.calitp_itp_id, _.calitp_url_number,
#               _.date == _.service_date,
#               _.trip_key, _.trip_id, _.route_id, _.direction_id,
#               _.shape_id, _.calitp_extracted_at, _.calitp_deleted_at)
#     # >> filter(_.route_id.isin(pch_routes))

#     >> collect()
#     )

In [11]:
# lbt_trips.to_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [12]:
lbt_trips = pd.read_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [13]:
# stop_times = (tbl.gtfs_schedule.stop_times()
#      >> filter(_.calitp_itp_id == 170)
#      # >> filter(_.trip_id == '9288623')
#      >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
#      >> arrange(_.stop_sequence)
#      >> collect()
#     )

In [14]:
# stop_times.to_parquet(f'{GCS_FILE_PATH}lbt_st.parquet')

In [15]:
stop_times = pd.read_parquet(f'{GCS_FILE_PATH}lbt_st.parquet')

In [16]:
# stops = (tbl.gtfs_schedule.stops()
#      >> filter(_.calitp_itp_id == 170)
#      # >> filter(_.trip_id == '9288623')
#      # >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
#      # >> arrange(_.stop_sequence)
#      >> select(_.calitp_itp_id, _.calitp_url_number, _.stop_id,
#               _.stop_lat, _.stop_lon, _.stop_name)
#      >> collect()
#     )

# stops = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
#                         crs='EPSG:4326').to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [17]:
# shared_utils.utils.geoparquet_gcs_export(stops, GCS_FILE_PATH, 'lbt_stops')

In [18]:
stops = gpd.read_parquet(f'{GCS_FILE_PATH}lbt_stops.parquet')

In [19]:
trips_positions_joined = (lbt_trips
                        # >> filter(_.route_id.isin(pch_routes))
                        >> inner_join(_, (lbt_positions >> rename(trip_id = 'vehicle_trip_id')),
                                      on= ['trip_id', 'calitp_itp_id', 'calitp_url_number'])
                       )

In [20]:
trips_positions_joined.vehicle_timestamp.max()

Timestamp('2021-12-01 11:15:59')

In [21]:
trips_positions_joined.vehicle_timestamp.min()

Timestamp('2021-11-30 15:54:58')

In [22]:
trips_positions_joined = gpd.GeoDataFrame(trips_positions_joined,
                                geometry=gpd.points_from_xy(trips_positions_joined.vehicle_position_longitude,
                                                            trips_positions_joined.vehicle_position_latitude),
                                crs=shared_utils.geography_utils.WGS84).to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [23]:
# lbt_routelines = shared_utils.geography_utils.make_routes_shapefile(['170'], ## LBT
#                                 shared_utils.geography_utils.CA_NAD83Albers)

In [24]:
# shared_utils.utils.geoparquet_gcs_export(lbt_routelines, GCS_FILE_PATH, 'lbt_routelines')

In [25]:
lbt_routelines = gpd.read_parquet(f'{GCS_FILE_PATH}lbt_routelines.parquet')

## Vehicle Positions Trip class

In [70]:
class VehiclePositionsTrip:
    '''Trip data and useful methods for analyzing GTFS-RT vehicle positions data'''
    
    def __init__(self, vp_gdf, shape_gdf):
        
        global vp_gdff
        vp_gdff = vp_gdf
        
        assert vp_gdf.crs == shared_utils.geography_utils.CA_NAD83Albers
        vp_gdf = vp_gdf >> distinct(_.trip_id, _.vehicle_timestamp, _keep_all=True)
        
        self.date = vp_gdf.date.iloc[0]
        self.trip_id = vp_gdf.trip_id.iloc[0]
        self.route_id = vp_gdf.route_id.iloc[0]
        self.shape_id = vp_gdf.shape_id.iloc[0]
        self.direction_id = vp_gdf.direction_id.iloc[0]
        self.entity_id = vp_gdf.entity_id.iloc[0]
        self.vehicle_id = vp_gdf.vehicle_id.iloc[0]
        self.calitp_itp_id = vp_gdf.calitp_itp_id.iloc[0]
        self.calitp_url_number = vp_gdf.calitp_url_number.iloc[0]
        self.vehicle_positions = vp_gdf >> select(_.vehicle_timestamp,
                                              _.header_timestamp,
                                              _.geometry)
        self._attach_shape(shape_gdf)
        
    def _attach_shape(self, shape_gdf):
        assert shape_gdf.crs == shared_utils.geography_utils.CA_NAD83Albers
        assert shape_gdf.calitp_itp_id.iloc[0] == self.calitp_itp_id
        self.shape = (shape_gdf
                        >> filter(_.shape_id == self.shape_id)
                        >> select(_.shape_id, _.geometry))
        self.vehicle_positions['shape_meters'] = (self.vehicle_positions.geometry
                                .apply(lambda x: self.shape.geometry.iloc[0].project(x)))
        self._linear_reference()
        
        origin = (self.vehicle_positions >> filter(_.shape_meters == _.shape_meters.min())
        ).geometry.iloc[0]
        destination = (self.vehicle_positions >> filter(_.shape_meters == _.shape_meters.max())
        ).geometry.iloc[0]
        self.direction = primary_cardinal_direction(origin, destination)
        
    def _linear_reference(self):
        
        self.vehicle_positions = self._shift_calculate(self.vehicle_positions)
        self.progressing_positions = self.vehicle_positions >> filter(_.progressed)
        ## check if positions have progressed from immediate previous point, but not previous point of forwards progression
        if not self.progressing_positions.shape_meters.is_monotonic:
            print(f'check location data for trip {self.trip_id}')
            self._fix_progression()
    
    def _fix_progression(self):
        
        self.progressing_positions = self._shift_calculate(self.progressing_positions)
        self.progressing_positions = self.progressing_positions >> filter(_.progressed)
        ## check if positions have progressed from immediate previous point, but not previous point of forwards progression
        if not self.progressing_positions.shape_meters.is_monotonic:
            # print(f'recheck location data for trip {self.trip_id}')
            self._fix_progression()
    
    def _shift_calculate(self, vehicle_positions):
        vehicle_positions['last_time'] = vehicle_positions.vehicle_timestamp.shift(1)
        vehicle_positions['last_loc'] = vehicle_positions.shape_meters.shift(1)
        vehicle_positions['secs_from_last'] = vehicle_positions.vehicle_timestamp - vehicle_positions.last_time
        vehicle_positions.secs_from_last = (vehicle_positions.secs_from_last
                                        .apply(lambda x: x.seconds))
        vehicle_positions['meters_from_last'] = (vehicle_positions.shape_meters
                                                      - vehicle_positions.last_loc)
        vehicle_positions['progressed'] = vehicle_positions['meters_from_last'] > 0 ## has the bus moved ahead?
        vehicle_positions['speed_from_last'] = (vehicle_positions.meters_from_last
                                                     / vehicle_positions.secs_from_last) ## meters/second
        return vehicle_positions
        
        
#     def position_at_time(self, dt): ## implement if/when needed
        
    def time_at_position(self, desired_position):
        
        global bounding_points
        
        try:
            next_point = (self.progressing_positions
                  >> filter(_.shape_meters > desired_position)
                  >> filter(_.shape_meters == _.shape_meters.min())
                 )
            prev_point = (self.progressing_positions
                  >> filter(_.shape_meters < desired_position)
                  >> filter(_.shape_meters == _.shape_meters.max())
                 )
            bounding_points = (prev_point.append(next_point).copy().reset_index(drop=True)
                    >> select(-_.secs_from_last, -_.meters_from_last, -_.speed_from_last)) ## drop in case bounding points are nonconsecutive
            secs_from_last = (bounding_points.loc[1].vehicle_timestamp - bounding_points.loc[0].vehicle_timestamp).seconds
            meters_from_last = bounding_points.loc[1].shape_meters - bounding_points.loc[0].shape_meters
            speed_from_last = meters_from_last / secs_from_last

            meters_position_to_next = bounding_points.loc[1].shape_meters - desired_position
            est_seconds_to_next = meters_position_to_next / speed_from_last
            est_td_to_next = dt.timedelta(seconds=est_seconds_to_next)
            est_dt = bounding_points.iloc[-1].vehicle_timestamp - est_td_to_next

            return est_dt
        except KeyError:
            print(f'insufficient bounding points for trip {self.trip_id}, location {desired_position}', end=': ')
            print(f'start/end of route?')
            return None
        
    def detailed_speed_map(self):
        
        gdf = self.vehicle_positions.copy()
        gdf['time'] = gdf.vehicle_timestamp.apply(lambda x: x.strftime('%H:%M:%S'))

        gdf = gdf >> select(_.geometry, _.time,
                            _.shape_meters, _.last_loc, _.speed_from_last)
        gdf['speed_mph'] = gdf.speed_from_last * MPH_PER_MPS
        
        gdf.geometry = gdf.apply(lambda x: shapely.ops.substring(self.shape.geometry.iloc[0],
                                                                x.last_loc,
                                                                x.shape_meters),
                                axis = 1)
        
        gdf.geometry = gdf.buffer(25)
        gdf = gdf.to_crs(shared_utils.geography_utils.WGS84)
        gdf = gdf >> filter(_.speed_mph > 0)
        gdf = gdf >> mutate(speed_mph = _.speed_mph.round(1),
                           shape_meters = _.shape_meters.round(0))
        
        gdf['shape_id'] = self.shape_id
        gdf['direction_id'] = self.direction_id
        gdf['trip_id'] = self.trip_id
        # display(gdf)
        
        if gdf.speed_mph.max() > 70:
            print(f'speed above 70 for trip {self.trip_id}, dropping')
            gdf = gdf >> filter(_.speed_mph < 70)

        colorscale = branca.colormap.step.RdYlGn_08.scale(vmin=gdf.speed_mph.min(), 
                     vmax=gdf.speed_mph.max())
        colorscale.caption = "Speed (miles per hour)"
        
        popup_dict = {
            "speed_mph": "Speed (miles per hour)",
            "shape_meters": "Distance along route (meters)",
            "shape_id": "Shape ID",
            "direction_id": "Direction ID",
            "trip_id": "Trip ID",
            "time": "Time"
        }
        
        g = shared_utils.map_utils.make_folium_choropleth_map(
            gdf,
            plot_col = 'speed_mph',
            popup_dict = popup_dict,
            tooltip_dict = popup_dict,
            colorscale = colorscale,
            fig_width = 1000, fig_height = 700,
            zoom = 13,
            centroid = [33.790, -118.154],
            title=f"Trip Speed Map (Route {self.route_id}, {self.direction}, PM Peak)", ##TODO time classification, remove hardcode
            highlight_function=lambda x: {
                'fillColor': '#DD1C77',
                "fillOpacity": 0.6,
            }
        )

        return g

#### Examples

In [71]:
example_trip = trips_positions_joined >> filter(_.trip_id == '9288623')

In [72]:
example_vp_trip = VehiclePositionsTrip(example_trip, lbt_routelines)

In [73]:
example_vp_trip.vehicle_positions.head(3)

,vehicle_timestamp,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
0,2021-11-30 17:02:12,2021-11-30 17:02:37,POINT (174550.467 -468602.675),74.936069,NaT,NaN,NaN,NaN,False,NaN
1,2021-11-30 17:03:13,2021-11-30 17:03:37,POINT (174199.373 -468606.652),426.501155,2021-11-30 17:02:12,74.936069,61.0,351.565087,True,5.763362
2,2021-11-30 17:04:13,2021-11-30 17:04:37,POINT (173950.148 -468607.933),676.459161,2021-11-30 17:03:13,426.501155,60.0,249.958006,True,4.165967


In [74]:
example_vp_trip.time_at_position(550) ## interpolate time at 550 meters from start of route (shape_meters in df^)

Timestamp('2021-11-30 17:03:42.644702')

In [44]:
# example_vp_trip.detailed_speed_map()

## General RT Analysis class

In [55]:
class RtAnalysis:
    '''Current top-level class for GTFS-RT analysis'''
    
    def __init__(self, trips_positions_joined, stop_times, stops, shape_gdf, trip_ids): ## trips_position_joined is temporary
        
        for df in (trips_positions_joined, stop_times, stops, shape_gdf):
            assert df.calitp_itp_id.nunique() == 1
            assert df.calitp_url_number.nunique() == 1
        
        self.trips_positions = trips_positions_joined >> filter(_.trip_id.isin(trip_ids))
        self.stop_times = stop_times >> filter(_.trip_id.isin(trip_ids))
        self.stops = stops
        self.st_geo = self.stops >> inner_join(_, self.stop_times, on = ['calitp_itp_id', 'calitp_url_number','stop_id'])
        self.shape_gdf = shape_gdf
        self.trip_ids = trip_ids
        self.generate_vp_trips()
        
    def generate_vp_trips(self):
        self.trip_vehicle_positions = {}
        for trip_id in self.trip_ids:
            self.trip_vehicle_positions[trip_id] = VehiclePositionsTrip(
                                            self.trips_positions >> filter(_.trip_id == trip_id),
                                            self.shape_gdf)
        
    def generate_delay_view(self, trip_ids = None):
        print('gdv called')

        if  type(trip_ids) == type(None): ## trip_ids could potentially be a list or pd.Series...
            trip_ids = self.trip_ids
        
        self._delay_view_inputs = trip_ids
        self.delay_view = gpd.GeoDataFrame()
        trips_processed = 0
        for trip_id in trip_ids:
            try:
                trip_rt_data = self.trip_vehicle_positions[trip_id]
                trip_st = (self.stop_times >> filter(_.trip_id == trip_id)).copy()
                trip_st_geo = (self.st_geo >> filter(_.trip_id == trip_id)).copy()
                trip_st_geo['route_id'] = trip_rt_data.route_id
                trip_st_geo['shape_id'] = trip_rt_data.shape_id
                trip_st_geo['direction_id'] = trip_rt_data.direction_id
                trip_st_geo['direction'] = trip_rt_data.direction
                trip_st_geo['shape_meters'] = (trip_st_geo.geometry
                                                .apply(lambda x: trip_rt_data.shape.project(x)))
                trip_st_geo['actual_time'] = (trip_st_geo.shape_meters
                                              .apply(lambda x: trip_rt_data.time_at_position(x)))
                trip_st_geo = trip_st_geo.dropna(subset=['actual_time'])
                trip_st_geo['arrival_time'] = trip_st_geo.apply(lambda x:
                                            dt.datetime.combine(x.actual_time.date(),
                                                                dt.datetime.strptime(x.arrival_time, '%H:%M:%S').time()),
                                                                axis = 1) ## format scheduled arrival times
                # _debug = trip_st_geo
                trip_st_geo['delay'] = trip_st_geo.actual_time - trip_st_geo.arrival_time
                trip_st_geo['date'] = trip_rt_data.date
                trip_view = trip_st_geo.dropna(subset=['delay']) >> arrange(_.arrival_time) >> select(
                                                                _.arrival_time, _.actual_time, _.delay,
                                                             _.stop_id, _.trip_id, _.shape_id, _.direction_id,
                                                            _.direction, _.stop_sequence, _.route_id,
                                                            _.shape_meters, _.date, _.geometry)
                self.delay_view = self.delay_view.append(trip_view)
                trips_processed += 1
                if trips_processed % 5 == 0:
                    print(f'{trips_processed} trips processed')
            except Exception as e:
                print(trip_id, e)
        self.delay_view = (self.delay_view >> arrange(_.stop_sequence, _.trip_id)).set_crs(self.st_geo.crs)
        return self.delay_view
    
    def generate_delay_summary(self, trip_ids = None):       

        if  type(trip_ids) == type(None): ## trip_ids could potentially be a list or pd.Series...
            trip_ids = self.trip_ids
        
        if hasattr(self, 'delay_view') and set(list(self.delay_view.trip_id.unique())) == set(list(trip_ids)):
        ## if delay view exists and matches request, use it
            stop_geos = self.delay_view >> select(_.stop_id, _.geometry) >> distinct(_.stop_id, _keep_all=True)
            self.delay_summary = (self.delay_view
                     >> group_by(_.stop_id, _.stop_sequence,)
                     >> summarize(avg_delay = _.delay.mean(), max_delay = _.delay.max())
                     # >> inner_join(_, stop_geos, on = 'stop_id')
                     >> arrange(_.stop_sequence)
                    )
            self.delay_summary = stop_geos >> inner_join(_, self.delay_summary, on = 'stop_id')
        else:
            self.generate_delay_view(trip_ids) ## generate new delay view if necessary
            return self.generate_delay_summary(trip_ids)
        return self.delay_summary
    
    def map_stop_delays(self, how = 'max',  trip_ids = None):
                
        if  type(trip_ids) == type(None): ## trip_ids could potentially be a list or pd.Series...
            trip_ids = self.trip_ids
        assert how in ['max', 'average']
        self.generate_delay_summary(trip_ids)
        
        gdf = self.delay_summary.copy()
        if how == 'max':
            gdf['delay_minutes'] = gdf.max_delay.apply(lambda x: x.seconds / 60)
        elif how == 'average':
            gdf['delay_minutes'] = gdf.avg_delay.apply(lambda x: x.seconds / 60)
        gdf['delay_minutes'] = gdf.delay_minutes.round(0)
        gdf = gdf >> select(_.stop_id, _.geometry, _.delay_minutes)
        gdf.geometry = gdf.buffer(50)
        
        gdf = gdf.to_crs(shared_utils.geography_utils.WGS84)

        colorscale = reversed_colormap(branca.colormap.step.RdYlGn_08.scale(vmin=0, 
                     vmax=gdf.delay_minutes.max()))
        colorscale.caption = "Delay (minutes)"
        
        popup_dict = {
            "delay_minutes": "Delay (minutes)",
            "stop_id": "Stop ID",
        }
        
        g = shared_utils.map_utils.make_folium_choropleth_map(
            gdf,
            plot_col = 'delay_minutes',
            popup_dict = popup_dict,
            tooltip_dict = popup_dict,
            colorscale = colorscale,
            fig_width = 1000, fig_height = 700,
            zoom = 13,
            centroid = [33.790, -118.154],
            title="Stop Delay Map"
        )

        return g

        
    def map_segment_speeds(self, how = 'high_delay', segments = 'stops', trip_ids = None): ##TODO split out segment speed view?

        if  type(trip_ids) == type(None): ## trip_ids could potentially be a list or pd.Series...
            trip_ids = self.trip_ids
        
        assert how in ['low_speeds', 'average']
        assert segments in ['stops', 'detailed']
        
        speed_calculators = {'low_speeds': _.speed_mph.quantile(.2), ## 20th percentile speed
                            'average': _.speed_mph.mean()} ## average speed
        
        if hasattr(self, 'delay_view') and set(list(self._delay_view_inputs)) == set(list(trip_ids)):
            
            all_stop_speeds = gpd.GeoDataFrame()
            for shape_id in self.delay_view.shape_id.unique():
                for direction_id in self.delay_view.direction_id.unique():
                    stop_speeds = (self.delay_view
                                 >> filter((_.shape_id == shape_id) & (_.direction_id == direction_id))
                                 >> group_by(_.trip_id)
                                 >> arrange(_.stop_sequence)
                                 >> mutate(seconds_from_last = (_.actual_time - _.actual_time.shift(1)).apply(lambda x: x.seconds))
                                 >> mutate(last_loc = _.shape_meters.shift(1))
                                 >> mutate(meters_from_last = (_.shape_meters - _.last_loc))
                                 >> mutate(speed_from_last = _.meters_from_last / _.seconds_from_last) 
                                 >> ungroup()
                                )
                    stop_speeds.geometry = stop_speeds.apply(
                        lambda x: shapely.ops.substring(
                                    self.trip_vehicle_positions[x.trip_id].shape.geometry.iloc[0],
                                    x.last_loc,
                                    x.shape_meters),
                                                    axis = 1)
                    stop_speeds = stop_speeds.dropna(subset=['last_loc']).set_crs(shared_utils.geography_utils.CA_NAD83Albers)

                    stop_speeds = (stop_speeds
                         >> mutate(speed_mph = _.speed_from_last * MPH_PER_MPS)
                         >> group_by(_.stop_sequence)
                         >> mutate(speed_mph = speed_calculators[how])
                         >> mutate(speed_mph = _.speed_mph.round(1))
                         >> mutate(shape_meters = _.shape_meters.round(0))
                         >> distinct(_.stop_sequence, _keep_all=True)
                         >> ungroup()
                         >> select(-_.arrival_time, -_.actual_time, -_.delay, -_.trip_id)
                        )
                    print(stop_speeds.shape_id.iloc[0], stop_speeds.shape)
                    if stop_speeds.speed_mph.max() > 70:
                        print(f'speed above 70 for shape {stop_speeds.shape_id.iloc[0]}, dropping')
                        stop_speeds = stop_speeds >> filter(_.speed_mph < 70)
                    all_stop_speeds = all_stop_speeds.append(stop_speeds)
            
            self.segment_speed_view = all_stop_speeds
            return self._generate_segment_map(how = how)
            
        else:
            self.generate_delay_view(trip_ids) ## generate new delay view if necessary
            return self.map_segment_speeds(how, segments, trip_ids = trip_ids)
        
    def _generate_segment_map(self, how, colorscale = None, size = [900, 550]):
        
        how_formatted = {'average': 'Average', 'low_speeds': '20th Percentile'}

        gdf = self.segment_speed_view >> select(-_.date)
        gdf.geometry = gdf.set_crs(shared_utils.geography_utils.CA_NAD83Albers).buffer(25)
        
        if not colorscale:
            colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=gdf.speed_mph.min(), 
             vmax=gdf.speed_mph.max())
            colorscale.caption = "Speed (miles per hour)"

        popup_dict = {
            "speed_mph": "Speed (miles per hour)",
            "shape_meters": "Distance along route (meters)",
            "route_id": "Route",
            "direction": "Direction",
            "shape_id": "Shape ID",
            "direction_id": "Direction ID",
            "stop_id": "Next Stop ID",
            "stop_sequence": "Next Stop Sequence"
        }

        g = shared_utils.map_utils.make_folium_choropleth_map(
            gdf,
            plot_col = 'speed_mph',
            popup_dict = popup_dict,
            tooltip_dict = popup_dict,
            colorscale = colorscale,
            fig_width = size[0], fig_height = size[1],
            zoom = 13,
            centroid = [33.790, -118.154],
            title=f"Long Beach Transit {how_formatted[how]} Bus Speeds Between Stops, Afternoon Peak",
            highlight_function=lambda x: {
                'fillColor': '#DD1C77',
                "fillOpacity": 0.6,
            }
        )

        return g  
    
    def route_coverage_summary(self):
        
        if hasattr(self, 'delay_view'):
            summary = (self.delay_view
             >> group_by(_.trip_id, _.shape_id, _.direction_id)
             >> summarize(min_meters = _.shape_meters.min(),
                         min_stop = _.stop_sequence.min(),
                         max_meters = _.shape_meters.max(),
                         max_stop = _.stop_sequence.max())
            )
            return summary
        else:
            self.generate_delay_view()
            return self.route_coverage_summary()

In [56]:
example_trips = (trips_positions_joined
 >> filter(_.route_id == '175', _.direction_id == '0') ## towards Villages at Cabrillo
 >> group_by(_.trip_id)
 >> summarize(max_time = _.vehicle_timestamp.max())
 >> arrange(_.max_time)
 >> head(6) ## afternoon/evening
)
example_trips

,trip_id,max_time
8,9288619,2021-11-30 16:37:23
9,9288621,2021-11-30 17:17:58
10,9288623,2021-11-30 17:56:04
11,9288625,2021-11-30 18:35:49
12,9288627,2021-11-30 19:24:12
13,9288629,2021-11-30 19:59:08


In [57]:
%%capture
## suppress warnings from location data inaccuracies
## generate test analysis with a small subset of LBT trips on a particular route, time, direction
test_analysis = RtAnalysis(trips_positions_joined, stop_times, stops, lbt_routelines, example_trips.trip_id)

In [58]:
%%capture
## suppress warnings from start/end of routes
all_delays = test_analysis.generate_delay_view()

In [59]:
all_delays.head(3)

,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,date,geometry
148,2021-11-30 16:58:10,2021-11-30 17:03:30.892511,0 days 00:05:20.892511,1027,9288623,1750012,0,Westbound,2,175,501.040761,2021-12-01,POINT (174124.462 -468604.478)
145,2021-11-30 17:39:10,2021-11-30 17:43:07.179287,0 days 00:03:57.179287,1027,9288625,1750012,0,Westbound,2,175,501.040761,2021-12-01,POINT (174124.462 -468604.478)
86,2021-11-30 17:00:17,2021-11-30 17:05:15.130531,0 days 00:04:58.130531,0540,9288623,1750012,0,Westbound,3,175,991.109291,2021-12-01,POINT (173921.407 -468897.107)


In [60]:
endpoint_delay_sum = (all_delays
 >> group_by(_.trip_id)
 >> filter(_.stop_sequence == _.stop_sequence.max())
 >> ungroup()
 >> mutate(total_delay = _.delay.sum())
)

In [61]:
endpoint_delay_sum

,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,date,geometry,total_delay
0,2021-11-30 16:58:00,2021-11-30 17:10:28.976334,0 days 00:12:28.976334,3150,9288621,1750012,0,Westbound,29,175,11480.641185,2021-12-01,POINT (164664.672 -467611.924),0 days 01:04:39.968770
1,2021-11-30 18:56:00,2021-11-30 19:20:48.778780,0 days 00:24:48.778780,3150,9288627,1750012,0,Westbound,29,175,11480.641185,2021-12-01,POINT (164664.672 -467611.924),0 days 01:04:39.968770
2,2021-11-30 17:34:39,2021-11-30 17:44:59.937535,0 days 00:10:20.937535,3049,9288623,1750012,0,Westbound,28,175,10745.097772,2021-12-01,POINT (164733.079 -467919.985),0 days 01:04:39.968770
3,2021-11-30 16:14:39,2021-11-30 16:28:44.013076,0 days 00:14:05.013076,3049,9288619,1750012,0,Westbound,28,175,10745.097772,2021-12-01,POINT (164733.079 -467919.985),0 days 01:04:39.968770
4,2021-11-30 18:14:39,2021-11-30 18:16:47.014459,0 days 00:02:08.014459,3049,9288625,1750012,0,Westbound,28,175,10745.097772,2021-12-01,POINT (164733.079 -467919.985),0 days 01:04:39.968770
5,2021-11-30 19:37:39,2021-11-30 19:38:27.248586,0 days 00:00:48.248586,3049,9288629,1750012,0,Westbound,28,175,10745.097772,2021-12-01,POINT (164733.079 -467919.985),0 days 01:04:39.968770


In [62]:
all_summary = test_analysis.generate_delay_summary()

In [63]:
all_summary.head(3)

,stop_id,geometry,stop_sequence,avg_delay,max_delay
0,1027,POINT (174124.462 -468604.478),2,0 days 00:04:39.035899,0 days 00:05:20.892511
1,0540,POINT (173921.407 -468897.107),3,0 days 00:04:00.527345,0 days 00:04:58.130531
2,0542,POINT (173328.363 -468647.513),4,0 days 00:01:40.985010250,0 days 00:03:24.379293


In [64]:
# test_analysis.map_stop_delays(how = 'average') ## to refine, add hover, legend, etc...

In [65]:
test_analysis.map_segment_speeds(how = 'low_speeds')

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


1750012 (27, 14)


In [251]:
# test_analysis.map_stop_delays()

## Sandbox

In [237]:
trips_positions_joined >> filter(_.direction_id == '0') >> count(_.trip_id)

,trip_id,n
0,9288152,204
1,9288153,354
2,9288154,259
3,9288155,247
4,9288156,272
...,...,...
633,9296926,61
634,9296927,103
635,9296928,103
636,9297176,68


In [238]:
ex3_trips = (trips_positions_joined
 >> filter(_.direction_id == '0') ## towards 
 # >> filter((_.vehicle_timestamp.apply(lambda x: x.hour < 20))
 #           & (_.vehicle_timestamp.apply(lambda x: x.hour > 16))) ## 4-8pm
 >> mutate(timestamp_hour = _.vehicle_timestamp.apply(lambda x: x.hour))
 >> filter((_.timestamp_hour < 20)
           & (_.timestamp_hour > 16)) ## 4-8pm >> group_by(_.trip_id)
 >> group_by(_.trip_id)
 >> summarize(max_time = _.vehicle_timestamp.max())
 >> arrange(_.max_time)
 # >> head(40) ## afternoon/evening
)
ex3_trips

,trip_id,max_time
0,9288170,2021-11-30 17:00:34
130,9289459,2021-11-30 17:01:25
121,9289404,2021-11-30 17:01:52
58,9288776,2021-11-30 17:02:20
190,9296915,2021-11-30 17:03:04
...,...,...
77,9288961,2021-11-30 19:59:52
166,9296637,2021-11-30 19:59:52
145,9289764,2021-11-30 19:59:54
7,9288179,2021-11-30 19:59:55


In [239]:
test_many = RtAnalysis(trips_positions_joined,
                           stop_times,
                           stops,
                           lbt_routelines,
                           ex3_trips.trip_id)

check location data for trip 9288776


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


check location data for trip 9288621


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


check location data for trip 9296673


...

/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

check location data for trip 9288630


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


check location data for trip 9288179


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [240]:
%%capture --no-stderr
test_many.generate_delay_view()

In [241]:
test_many.trip_ids.nunique()

195

In [242]:
test_many.delay_view.trip_id.nunique()

193

In [243]:
test_many.delay_view.shape_id.nunique()

39

In [244]:
m = test_many.map_segment_speeds(how = 'average')

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


920111 (64, 12)


/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


930147 (74, 12)


/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


...

460069 (25, 12)


/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


10069 (7, 12)


/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


20008 (10, 12)


In [245]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, 
 vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [246]:
m = test_many._generate_segment_map(colorscale)

In [247]:
type(m)

branca.element.Figure

In [357]:
# m

In [248]:
# m

In [276]:
m.save('lbt_speed_map.html')

In [75]:
m = example_vp_trip.detailed_speed_map()

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [76]:
m

In [69]:
m.save('lbt_detailled_speeds.html')